In [16]:
import os


def convert_mem_to_sv():
    int_bits = 6
    frac_bits = range(1,17)
    N_classes = 10
    for N in range(N_classes):
       
        # Create sv-file with ROM description
        path = f"ROM_files\sv\\"
        if not os.path.exists(path):
            os.makedirs(path)
            
        filename = path + f'ROM_weights_{N}.sv'
        file_out = open(filename, mode='w')

        file_out.write(f"`timescale 1ns / 1ps \n \n")
        file_out.write(f"////////////////////////////////////////////////////////////////////////////////// \n")
        file_out.write(f"// WEIGHT MEMORY (ROM) \n")
        file_out.write(f"////////////////////////////////////////////////////////////////////////////////// \n")

        file_out.write(f"module ROM_weights_{N} #( \n")
        file_out.write(f"    parameter int WGHT_INT = 6, // integer part \n")
        file_out.write(f"    parameter int WGHT_FRC = 16 // fractional part \n")
        file_out.write(f")( \n")
        file_out.write(f"    input logic clk, // clock \n") 
        file_out.write(f"    input logic [9:0] address,\n") 
        file_out.write(f"    output [WGHT_INT + WGHT_FRC-1:0] dout \n") 
        file_out.write(f"); \n\n") 
        file_out.write(f'(* rom_style = "block" *) reg [WGHT_INT + WGHT_FRC-1:0] data;\n') 

        
        # Open mem-file with memory content
        for f_bits in frac_bits:
            filename = f'ROM_files\mem\\format_q{int_bits}_{f_bits}\\weight_q{int_bits}_{f_bits}_N{N}.mem'
            file = open(filename, mode='r')

            file_out.write(f"generate \n")
            file_out.write(f"  if (WGHT_FRC == {f_bits}) begin  \n")
            
            file_out.write(f"    always @(posedge clk) \n") 
            file_out.write(f"      begin \n") 
            file_out.write(f"        case(address) \n") 

            i = 0
            for content in file:
                file_out.write(f"        10'b{i:010b}: data <= {int_bits+f_bits}'h{content[:-1]}; \n")
                i = i + 1

            file_out.write(f"      endcase \n")
            file_out.write(f"    end \n")
            file_out.write(f"  end \n")
            file_out.write(f"endgenerate \n")
            file.close()
            
        file_out.write(f"assign dout = data; \n")
        file_out.write(f"endmodule\n")

        file_out.close()

In [17]:
convert_mem_to_sv()